<a href="https://colab.research.google.com/github/BhojRajBist/BhojRajBist/blob/main/statistical_api_sh_docs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests requests-oauthlib

In [6]:
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session


# Your Sentinel Hub Client Credentials (replace with your actual values)
client_id = ''
client_secret = ''

# Create a session
client = BackendApplicationClient(client_id=client_id)
oauth = OAuth2Session(client=client)

# Get token for the session
token = oauth.fetch_token(
    token_url='https://services.sentinel-hub.com/auth/realms/main/protocol/openid-connect/token',
    client_secret=client_secret,
    include_client_id=True
)

# Define the compliance hook to check for valid status codes
def sentinelhub_compliance_hook(response):
    response.raise_for_status()  # Raise an error for bad status codes
    return response

# Register the compliance hook to handle the access token response
oauth.register_compliance_hook("access_token_response", sentinelhub_compliance_hook)

# Test the authentication by making a request to the Sentinel Hub API
try:
    # Example API request (Sentinel Hub configuration endpoint)
    response = oauth.get("https://services.sentinel-hub.com/configuration/v1/wms/instances")
    print(response.json())  # Output the response from Sentinel Hub (in JSON format)
except Exception as e:
    print(f"An error occurred: {e}")


InvalidClientError: (invalid_client) Invalid client or Invalid client credentials

## Statistics for one single-band output on a given day

In [3]:
evalscript = """
//VERSION=3
function setup() {
  return {
    input: [{
      bands: [
        "B04",
        "dataMask"
      ]
    }],
    output: [
      {
        id: "output_B04",
        bands: 1,
        sampleType: "FLOAT32"
      },
      {
        id: "dataMask",
        bands: 1
      }]
  }
}
function evaluatePixel(samples) {
    return {
        output_B04: [samples.B04],
        dataMask: [samples.dataMask]
        }
}
"""

stats_request = {
  "input": {
    "bounds": {
      "bbox": [414315, 4958219, 414859, 4958819],
    "properties": {
        "crs": "http://www.opengis.net/def/crs/EPSG/0/32633"
        }
    },
    "data": [
      {
        "type": "sentinel-2-l2a",
        "dataFilter": {
            "mosaickingOrder": "leastRecent"
        },
      }
    ]
  },
  "aggregation": {
    "timeRange": {
            "from": "2020-07-04T00:00:00Z",
            "to": "2020-07-05T00:00:00Z"
      },
    "aggregationInterval": {
        "of": "P1D"
    },
    "evalscript": evalscript,
    "resx": 10,
    "resy": 10
  }
}

headers = {
  'Content-Type': 'application/json',
  'Accept': 'application/json'
}

url = "https://services.sentinel-hub.com/api/v1/statistics"
response = oauth.request("POST", url=url , headers=headers, json=stats_request)
sh_statistics = response.json()
sh_statistics

{'data': [{'interval': {'from': '2020-07-04T00:00:00Z',
    'to': '2020-07-05T00:00:00Z'},
   'outputs': {'output_B04': {'bands': {'B0': {'stats': {'min': 0.07970000058412552,
        'max': 0.30959999561309814,
        'mean': 0.11471141986778864,
        'stDev': 0.034298170449733226,
        'sampleCount': 3240,
        'noDataCount': 0}}}}}}],
 'status': 'OK'}

## Statistics, histogram and percentiles for one single-band output

In [4]:
evalscript = """
//VERSION=3
function setup() {
  return {
    input: [{
      bands: [
        "B04",
        "dataMask"
      ]
    }],
    output: [
      {
        id: "output_B04",
        bands: 1,
        sampleType: "FLOAT32"
      },
      {
        id: "dataMask",
        bands: 1
      }]
  }
}
function evaluatePixel(samples) {
    return {
        output_B04: [samples.B04],
        dataMask: [samples.dataMask]
        }
}
"""

stats_request = {
  "input": {
    "bounds": {
      "bbox": [414315, 4958219, 414859, 4958819],
    "properties": {
        "crs": "http://www.opengis.net/def/crs/EPSG/0/32633"
        }
    },
    "data": [
      {
        "type": "sentinel-2-l2a",
        "dataFilter": {
            "mosaickingOrder": "leastRecent"
        },
      }
    ]
  },
  "aggregation": {
    "timeRange": {
            "from": "2020-07-04T00:00:00Z",
            "to": "2020-07-05T00:00:00Z"
      },
    "aggregationInterval": {
        "of": "P1D"
    },
    "evalscript": evalscript,
    "resx": 10,
    "resy": 10
  },
  "calculations": {
    "default": {
      "histograms": {
        "default": {
          "nBins": 5,
          "lowEdge": 0.0,
          "highEdge": 0.3
        }
      },
      "statistics": {
        "default": {
          "percentiles": {
            "k": [ 33, 50, 75, 90 ]
          }
        }
      }
    }
  }
}

headers = {
  'Content-Type': 'application/json',
  'Accept': 'application/json'
}

url = "https://services.sentinel-hub.com/api/v1/statistics"

response = oauth.request("POST", url=url , headers=headers, json=stats_request)
sh_statistics = response.json()
sh_statistics

{'data': [{'interval': {'from': '2020-07-04T00:00:00Z',
    'to': '2020-07-05T00:00:00Z'},
   'outputs': {'output_B04': {'bands': {'B0': {'stats': {'min': 0.07970000058412552,
        'max': 0.30959999561309814,
        'mean': 0.11471141986778864,
        'stDev': 0.034298170449733226,
        'sampleCount': 3240,
        'noDataCount': 0,
        'percentiles': {'33.0': 0.09709999710321426,
         '75.0': 0.11940000206232071,
         '90.0': 0.16040000319480896,
         '50.0': 0.10360000282526016}},
       'histogram': {'bins': [{'lowEdge': 0.0, 'highEdge': 0.06, 'count': 0},
         {'lowEdge': 0.06, 'highEdge': 0.12, 'count': 2458},
         {'lowEdge': 0.12, 'highEdge': 0.18, 'count': 558},
         {'lowEdge': 0.18, 'highEdge': 0.24, 'count': 177},
         {'lowEdge': 0.24, 'highEdge': 0.3, 'count': 44}],
        'overflowCount': 3,
        'underflowCount': 0}}}}}}],
 'status': 'OK'}

## Statistics for one single-band output for two months with 10 days aggregation period

In [5]:
evalscript = """
//VERSION=3
function setup() {
  return {
    input: [{
      bands: [
        "B04",
        "dataMask"
      ]
    }],
    output: [
      {
        id: "output_B04",
        bands: 1,
        sampleType: "FLOAT32"
      },
      {
        id: "dataMask",
        bands: 1
      }]
  }
}
function evaluatePixel(samples) {
    return {
        output_B04: [samples.B04],
        dataMask: [samples.dataMask]
        }
}
"""

stats_request = {
  "input": {
   "bounds": {
      "bbox": [414315, 4958219, 414859, 4958819],
    "properties": {
        "crs": "http://www.opengis.net/def/crs/EPSG/0/32633"
        }
    },
    "data": [
      {
        "type": "sentinel-2-l2a",
        "dataFilter": {
            "mosaickingOrder": "leastRecent"
        }
      }
    ]
  },
  "aggregation": {
    "timeRange": {
            "from": "2020-06-01T00:00:00Z",
            "to": "2020-07-31T00:00:00Z"
      },
    "aggregationInterval": {
        "of": "P10D"
    },
    "evalscript": evalscript,
    "resx": 10,
    "resy": 10
  }
}
headers = {
  'Content-Type': 'application/json',
  'Accept': 'application/json'
}
url = "https://services.sentinel-hub.com/api/v1/statistics"

response = oauth.request("POST", url=url , headers=headers, json=stats_request)
sh_statistics = response.json()
sh_statistics

{'data': [{'interval': {'from': '2020-06-01T00:00:00Z',
    'to': '2020-06-11T00:00:00Z'},
   'outputs': {'output_B04': {'bands': {'B0': {'stats': {'min': 0.7892000079154968,
        'max': 0.8303999900817871,
        'mean': 0.804223583473102,
        'stDev': 0.0067066009561434865,
        'sampleCount': 3240,
        'noDataCount': 0}}}}}},
  {'interval': {'from': '2020-06-11T00:00:00Z', 'to': '2020-06-21T00:00:00Z'},
   'outputs': {'output_B04': {'bands': {'B0': {'stats': {'min': 0.016300000250339508,
        'max': 0.5956000089645386,
        'mean': 0.06240126554233315,
        'stDev': 0.06266500670629409,
        'sampleCount': 3240,
        'noDataCount': 0}}}}}},
  {'interval': {'from': '2020-06-21T00:00:00Z', 'to': '2020-07-01T00:00:00Z'},
   'outputs': {'output_B04': {'bands': {'B0': {'stats': {'min': 0.026000000536441803,
        'max': 0.43799999356269836,
        'mean': 0.06872379640174772,
        'stDev': 0.056520330692016944,
        'sampleCount': 3240,
        'noD

## Percentage of cloudy pixels for selected area of interest

In [ ]:
evalscript = """
//VERSION=3
function setup() {
  return {
    input: [{
      bands: [
        "CLM",
        "dataMask"
      ]
    }],
    output: [
      {
        id: "data",
        bands: 1
      },
      {
        id: "dataMask",
        bands: 1
      }]
  }
}
function evaluatePixel(samples) {
    return {
        data: [samples.CLM],
        dataMask: [samples.dataMask]
        }
}
"""

stats_request = {
  "input": {
   "bounds": {
      "bbox": [
          413307.629466,
          4957434.513693,
          415152.151806,
          4958814.807431
        ],
    "properties": {
        "crs": "http://www.opengis.net/def/crs/EPSG/0/32633"
        }
    },
    "data": [
      {
        "type": "sentinel-2-l2a",
        "dataFilter": {
            "mosaickingOrder": "leastRecent"
        }
      }
    ]
  },
  "aggregation": {
    "timeRange": {
            "from": "2020-11-01T00:00:00Z",
            "to": "2020-12-31T00:00:00Z"
      },
    "aggregationInterval": {
        "of": "P1D"
    },
    "evalscript": evalscript,
    "resx": 10,
    "resy": 10
  }
}

headers = {
  'Content-Type': 'application/json',
   'Accept': 'application/json'
}

url = "https://services.sentinel-hub.com/api/v1/statistics"
response = oauth.request("POST", url=url, headers=headers, json=stats_request)
sh_statistics = response.json()
sh_statistics

{'data': [{'interval': {'from': '2020-11-01T00:00:00Z',
    'to': '2020-11-02T00:00:00Z'},
   'outputs': {'data': {'bands': {'B0': {'stats': {'min': 1.0,
        'max': 1.0,
        'mean': 1.0,
        'stDev': 0.0,
        'sampleCount': 25392,
        'noDataCount': 0}}}}}},
  {'interval': {'from': '2020-11-06T00:00:00Z', 'to': '2020-11-07T00:00:00Z'},
   'outputs': {'data': {'bands': {'B0': {'stats': {'min': 0.0,
        'max': 0.0,
        'mean': 0.0,
        'stDev': 0.0,
        'sampleCount': 25392,
        'noDataCount': 0}}}}}},
  {'interval': {'from': '2020-11-11T00:00:00Z', 'to': '2020-11-12T00:00:00Z'},
   'outputs': {'data': {'bands': {'B0': {'stats': {'min': 0.0,
        'max': 0.0,
        'mean': 0.0,
        'stDev': 0.0,
        'sampleCount': 25392,
        'noDataCount': 0}}}}}},
  {'interval': {'from': '2020-11-21T00:00:00Z', 'to': '2020-11-22T00:00:00Z'},
   'outputs': {'data': {'bands': {'B0': {'stats': {'min': 0.0,
        'max': 0.0,
        'mean': 0.0,
    

In [ ]:
dates_without_clouds = [(data["interval"], int(100 * data["outputs"]["data"]['bands']['B0']['stats']['mean']) ) for data in sh_statistics["data"]]

for item in dates_without_clouds:
    print( item )

({'from': '2020-11-01T00:00:00Z', 'to': '2020-11-02T00:00:00Z'}, 100)
({'from': '2020-11-06T00:00:00Z', 'to': '2020-11-07T00:00:00Z'}, 0)
({'from': '2020-11-11T00:00:00Z', 'to': '2020-11-12T00:00:00Z'}, 0)
({'from': '2020-11-21T00:00:00Z', 'to': '2020-11-22T00:00:00Z'}, 0)
({'from': '2020-11-26T00:00:00Z', 'to': '2020-11-27T00:00:00Z'}, 31)
({'from': '2020-12-01T00:00:00Z', 'to': '2020-12-02T00:00:00Z'}, 28)
({'from': '2020-12-06T00:00:00Z', 'to': '2020-12-07T00:00:00Z'}, 100)
({'from': '2020-12-11T00:00:00Z', 'to': '2020-12-12T00:00:00Z'}, 98)
({'from': '2020-12-16T00:00:00Z', 'to': '2020-12-17T00:00:00Z'}, 100)
({'from': '2020-12-21T00:00:00Z', 'to': '2020-12-22T00:00:00Z'}, 100)
({'from': '2020-12-26T00:00:00Z', 'to': '2020-12-27T00:00:00Z'}, 15)


## Basic statistics of NDVI with water pixels excluded (custom output dataMask)

In [ ]:
evalscript = """
//VERSION=3
function setup() {
  return {
    input: [{
      bands: [
        "B04",
        "B08",
        "SCL",
        "dataMask"
      ]
    }],
    output: [
      {
        id: "data",
        bands: 1
      },
      {
        id: "dataMask",
        bands: 1
      }]
  }
}

function evaluatePixel(samples) {
    let ndvi = (samples.B08 - samples.B04)/(samples.B08 + samples.B04)

    var validNDVIMask = 1
    if (samples.B08 + samples.B04 == 0 ){
        validNDVIMask = 0
    }

    var noWaterMask = 1
    if (samples.SCL == 6 ){
        noWaterMask = 0
    }

    return {
        data: [ndvi],
        // Exclude nodata pixels, pixels where ndvi is not defined and water pixels from statistics:
        dataMask: [samples.dataMask * validNDVIMask * noWaterMask]
    }
}
"""


stats_request = {
  "input": {
   "bounds": {
      "geometry": {
          "type": "Polygon",
          "coordinates": [
            [
              [
                458085.878866,
                5097236.833044
              ],
              [
                457813.834156,
                5096808.351383
              ],
              [
                457979.897062,
                5096313.767184
              ],
              [
                458146.639373,
                5096405.411294
              ],
              [
                458085.878866,
                5097236.833044
              ]
            ]
          ]
        },
    "properties": {
        "crs": "http://www.opengis.net/def/crs/EPSG/0/32633"
        }
    },
    "data": [
      {
        "type": "sentinel-2-l2a",
        "dataFilter": {
            "mosaickingOrder": "leastCC"
        }
      }
    ]
  },
  "aggregation": {
    "timeRange": {
        "from": "2020-01-01T00:00:00Z",
        "to": "2020-12-31T00:00:00Z"
      },
    "aggregationInterval": {
        "of": "P30D"
    },
    "evalscript": evalscript,
    "resx": 10,
    "resy": 10
  }
}

headers = {
  'Content-Type': 'application/json',
  'Accept': 'application/json'
}

url = "https://services.sentinel-hub.com/api/v1/statistics"

response = oauth.request("POST", url=url, headers=headers, json=stats_request)
sh_statistics = response.json()
sh_statistics

{'data': [{'interval': {'from': '2020-01-01T00:00:00Z',
    'to': '2020-01-31T00:00:00Z'},
   'outputs': {'data': {'bands': {'B0': {'stats': {'min': 0.24306687712669373,
        'max': 0.6244725584983826,
        'mean': 0.4123224201824293,
        'stDev': 0.055874589607421886,
        'sampleCount': 3036,
        'noDataCount': 1192}}}}}},
  {'interval': {'from': '2020-01-31T00:00:00Z', 'to': '2020-03-01T00:00:00Z'},
   'outputs': {'data': {'bands': {'B0': {'stats': {'min': 0.2451941967010498,
        'max': 0.4233206510543823,
        'mean': 0.3160828609431641,
        'stDev': 0.0280772593636271,
        'sampleCount': 3036,
        'noDataCount': 1192}}}}}},
  {'interval': {'from': '2020-03-01T00:00:00Z', 'to': '2020-03-31T00:00:00Z'},
   'outputs': {'data': {'bands': {'B0': {'stats': {'min': 0.4236144721508026,
        'max': 0.8021259307861328,
        'mean': 0.5844831434836089,
        'stDev': 0.05766820795482124,
        'sampleCount': 3036,
        'noDataCount': 1192}}}}}

## Statistics of maximum monthly NDVI for a parcel in 2020

In [ ]:
evalscript = """
//VERSION=3
function setup() {
  return {
    input: [{
      bands: [
        "B04",
        "B08",
        "SCL",
        "dataMask"
      ]
    }],
    mosaicking: "ORBIT",
    output: [
      {
        id: "data",
        bands: ["monthly_max_ndvi"]
      },
      {
        id: "dataMask",
        bands: 1
      }]
  }
}

function evaluatePixel(samples) {
    var max = 0;
    var hasData = 0;
    for (var i=0;i<samples.length;i++) {
      if (samples[i].dataMask == 1 && samples[i].SCL != 6 && samples[i].B04+samples[i].B08 != 0 ){
        hasData = 1
        var ndvi = (samples[i].B08 - samples[i].B04)/(samples[i].B08 + samples[i].B04);
        max = ndvi > max ? ndvi:max;
      }
    }

    return {
        data: [max],
        dataMask: [hasData]
    }
}
"""

stats_request = {
  "input": {
   "bounds": {
      "geometry": {
          "type": "Polygon",
          "coordinates": [
            [
              [
                458085.878866,
                5097236.833044
              ],
              [
                457813.834156,
                5096808.351383
              ],
              [
                457979.897062,
                5096313.767184
              ],
              [
                458146.639373,
                5096405.411294
              ],
              [
                458085.878866,
                5097236.833044
              ]
            ]
          ]
        },
    "properties": {
        "crs": "http://www.opengis.net/def/crs/EPSG/0/32633"
        }
    },
    "data": [
      {
        "type": "sentinel-2-l2a",
        "dataFilter": {
            "mosaickingOrder": "leastCC"
        }
      }
    ]
  },
  "aggregation": {
    "timeRange": {
            "from": "2020-01-01T00:00:00Z",
            "to": "2021-01-01T00:00:00Z"
      },
    "aggregationInterval": {
        "of": "P1M"
    },
    "evalscript": evalscript,
    "resx": 10,
    "resy": 10
  }
}

headers = {
  'Content-Type': 'application/json',
   'Accept': 'application/json'
}

url = "https://services.sentinel-hub.com/api/v1/statistics"

response = oauth.request("POST", url=url, headers=headers, json=stats_request)
sh_statistics = response.json()
sh_statistics

{'data': [{'interval': {'from': '2020-01-01T00:00:00Z',
    'to': '2020-02-01T00:00:00Z'},
   'outputs': {'data': {'bands': {'monthly_max_ndvi': {'stats': {'min': 0.4755639135837555,
        'max': 0.881286084651947,
        'mean': 0.6396090604381046,
        'stDev': 0.06844923487502963,
        'sampleCount': 3036,
        'noDataCount': 1192}}}}}},
  {'interval': {'from': '2020-02-01T00:00:00Z', 'to': '2020-03-01T00:00:00Z'},
   'outputs': {'data': {'bands': {'monthly_max_ndvi': {'stats': {'min': 0.3580246865749359,
        'max': 0.8721038103103638,
        'mean': 0.5956351390500386,
        'stDev': 0.07367438999713516,
        'sampleCount': 3036,
        'noDataCount': 1192}}}}}},
  {'interval': {'from': '2020-03-01T00:00:00Z', 'to': '2020-04-01T00:00:00Z'},
   'outputs': {'data': {'bands': {'monthly_max_ndvi': {'stats': {'min': 0.4486735761165619,
        'max': 0.8021259307861328,
        'mean': 0.5871563556072766,
        'stDev': 0.057052289003643133,
        'sampleCount

## Multiple outputs with different dataMasks, multi-band output with custom bands' names and different histogram types

In [ ]:
evalscript = """
//VERSION=3
function setup() {
  return {
    input: [{
      bands: [
        "B04",
        "B08",
        "SCL",
        "dataMask"
      ]
    }],
    output: [
      {
        id: "output_my_bands",
        bands: ["only_band_B04", "only_band_B08"],
        sampleType: "FLOAT32"
      },
      {
        id: "output_my_indices",
        bands: 1,
        sampleType: "FLOAT32"
      },
      {
        id: "output_scl",
        bands: 1,
        sampleType: "UINT8"
      },
      {
        id: "dataMask",
        bands: ["output_my_bands", "output_my_indices"]
      }]
  }
}
function evaluatePixel(samples) {
    let ndvi = (samples.B08 - samples.B04)/(samples.B08 + samples.B04)

    var validNDVIMask = 1
    if (samples.B08 + samples.B04 == 0 ){
        validNDVIMask = 0
    }

    var noWaterMask = 1
    if (samples.SCL == 6 ){
        noWaterMask = 0
    }

    return {
        output_my_bands: [samples.B04, samples.B08],
        output_my_indices: [ndvi],
        output_scl: [samples.SCL],
        dataMask: [samples.dataMask, samples.dataMask * noWaterMask * validNDVIMask]
    }
}
"""

stats_request = {
  "input": {
   "bounds": {
      "bbox": [414315, 4958219, 414859, 4958819],
      "properties": {
        "crs": "http://www.opengis.net/def/crs/EPSG/0/32633"
      }
    },
    "data": [
      {
        "type": "sentinel-2-l2a",
        "dataFilter": {
            "mosaickingOrder": "leastRecent"
        }
      }
    ]
  },
  "aggregation": {
    "timeRange": {
            "from": "2020-07-01T00:00:00Z",
            "to": "2020-07-15T00:00:00Z"
      },
    "aggregationInterval": {
        "of": "P5D"
    },
    "evalscript": evalscript,
    "resx": 20,
    "resy": 20
  },
  "calculations": {
    "output_my_bands": {
      "histograms": {
        "only_band_B08": {
          "nBins": 3,
          "lowEdge": 0.0,
          "highEdge": 0.3
        }
      },
      "statistics": {
        "only_band_B04": {
          "percentiles": {
            "k": [33, 66,100],
          }
        }
      }
    },
    "output_scl": {
      "histograms": {
        "default": {
          "bins": [0,1,2,3,4,5,6,7,8,9,10,11]
        }
      }
    },
    "default": {
      "histograms": {
        "default": {
          "binWidth": 0.05,
          "lowEdge": 0.0
        }
      }
    }
  }
}

headers = {
  'Content-Type': 'application/json',
  'Accept': 'application/json'
}

url = "https://services.sentinel-hub.com/api/v1/statistics"

response = oauth.request("POST", url=url , headers=headers, json=stats_request)
sh_statistics = response.json()
sh_statistics

{'data': [{'interval': {'from': '2020-07-01T00:00:00Z',
    'to': '2020-07-06T00:00:00Z'},
   'outputs': {'output_my_bands': {'bands': {'only_band_B04': {'stats': {'min': 0.0803999975323677,
        'max': 0.2939999997615814,
        'mean': 0.11451061716602186,
        'stDev': 0.032769790113614555,
        'sampleCount': 810,
        'noDataCount': 0,
        'percentiles': {'33.0': 0.09719999879598618,
         '66.0': 0.11169999837875366,
         '100.0': 0.2939999997615814}}},
      'only_band_B08': {'stats': {'min': 0.0860000029206276,
        'max': 0.34290000796318054,
        'mean': 0.16518679009175594,
        'stDev': 0.07128630441809644,
        'sampleCount': 810,
        'noDataCount': 0},
       'histogram': {'bins': [{'lowEdge': 0.0,
          'highEdge': 0.09999999999999999,
          'count': 199},
         {'lowEdge': 0.09999999999999999,
          'highEdge': 0.19999999999999998,
          'count': 270},
         {'lowEdge': 0.19999999999999998, 'highEdge': 0.3, '

## Statistics for Sentinel-1

In [ ]:
evalscript = """
//VERSION=3
function setup() {
  return {
    input: [{
      bands: [
        "VV",
        "dataMask"
      ]
    }],
    output: [
      {
        id: "output_VV",
        bands: 1,
        sampleType: "FLOAT32"
      },
      {
        id: "dataMask",
        bands: 1
      }]
  }
}
function evaluatePixel(samples) {
    return {
        output_VV: [samples.VV],
        dataMask: [samples.dataMask]
        }
}
"""

stats_request = {
  "input": {
    "bounds": {
      "bbox": [414315, 4958219, 414859, 4958819],
    "properties": {
        "crs": "http://www.opengis.net/def/crs/EPSG/0/32633"
        }
    },
    "data": [
      {
        "type": "sentinel-1-grd",
        "dataFilter": {
        }
      }
    ]
  },
  "aggregation": {
    "timeRange": {
            "from": "2020-07-01T00:00:00Z",
            "to": "2020-07-10T00:00:00Z"
      },
    "aggregationInterval": {
        "of": "P5D"
    },
    "evalscript": evalscript,
    "resx": 10,
    "resy": 10
  }
}
headers = {
  'Content-Type': 'application/json',
  'Accept': 'application/json'
}
url = "https://services.sentinel-hub.com/api/v1/statistics"
response = oauth.request("POST", url=url , headers=headers, json=stats_request)
sh_statistics = response.json()
sh_statistics

{'data': [{'interval': {'from': '2020-07-01T00:00:00Z',
    'to': '2020-07-06T00:00:00Z'},
   'outputs': {'output_VV': {'bands': {'B0': {'stats': {'min': 0.0,
        'max': 0.4447733759880066,
        'mean': 0.046840328479290934,
        'stDev': 0.05487441687888816,
        'sampleCount': 3240,
        'noDataCount': 0}}}}}}],
 'status': 'OK'}

## Statistics of NDVI using Sentinel-2 L2A as the source of NDVI and Sentinel-1 GRD VV channel as the mask of water bodies

In [ ]:
evalscript = """
//VERSION=3
function setup() {
  return {
    input: [
      // Specify input bands using the "id" of datasource set in the payload under data parameter
      {datasource: "s2", bands: ["B04", "B08", "dataMask"]},
      {datasource: "s1", bands: ["VV", "dataMask"]}
    ],
    output: [
      {
        id: "ndvi",
        bands: 1
      },
      {
        id: "dataMask",
        bands: 1
      }],
    mosaicking: "SIMPLE"
  };
}

function evaluatePixel(samples) {
  let ndvi = (samples.s2[0].B08 - samples.s2[0].B04) / (samples.s2[0].B08+samples.s2[0].B04);

  // Create a mask for invalid ndvi value
  let validNDVIMask = 1;
  if (!isFinite(ndvi)) {
    validNDVIMask = 0;
  }

  // Create a mask for water
  // The threshold comes from the result of exploring river flooding during the winter of 2020/21 on the River Severn in the United Kingdom
  // (https://medium.com/euro-data-cube/exploring-time-and-space-a-guide-to-accessing-analysing-and-visualising-data-in-the-euro-data-e4a46f2bb55b)
  let noWaterMask = 1;
  if (toDB(samples.s1[0].VV) <= -20) {
    noWaterMask = 0;
  }
  return {
      ndvi: [ndvi],
      // Combine all the masks
      dataMask: [samples.s2[0].dataMask * samples.s1[0].dataMask * validNDVIMask * noWaterMask]
  };
}

function toDB(input){
  return 10 * Math.log(input)/Math.LN10;
}
"""
stats_request = {
  "input": {
    "bounds": {
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [16.72617,47.713689],
            [16.72617,47.655444],
            [16.816292,47.655444],
            [16.816292,47.713689],
            [16.72617,47.713689]
          ]
        ]
      }
    },
    "data": [
      {
        "dataFilter": {},
        "id": "s2",
        "type": "sentinel-2-l2a"
      },
      {
        "dataFilter": {
          "resolution": "HIGH",
          "acquisitionMode": "IW",
          "polarization": "DV"
        },
        "processing": {
          "backCoeff": "GAMMA0_TERRAIN",
          "orthorectify": "true",
          "demInstance": "MAPZEN",
          "speckleFilter": {
            "type": "LEE",
            "windowSizeX": 5,
            "windowSizeY": 5
          }
        },
        "id": "s1",
        "type": "sentinel-1-grd"
      }
    ]
  },
  "aggregation": {
    "timeRange": {
      "from": "2021-08-08T00:00:00Z",
      "to": "2021-08-11T23:59:59Z"
    },
    "aggregationInterval": {
      "of": "P1D"
    },
    "resx": 0.00009,
    "resy": 0.00009,
    "evalscript": evalscript
  },
  "calculations": {
    "default": {}
  }
}

headers = {
  'Content-Type': 'application/json',
  'Accept': 'application/json'
}

url = "https://services.sentinel-hub.com/api/v1/statistics"
response = oauth.request("POST", url=url, headers=headers, json=stats_request)
sh_statistics = response.json()
sh_statistics

{'data': [{'interval': {'from': '2021-08-08T00:00:00Z',
    'to': '2021-08-09T00:00:00Z'},
   'outputs': {'ndvi': {'bands': {'B0': {'stats': {'min': -0.6206604838371277,
        'max': 0.8291770815849304,
        'mean': 0.22080027097811286,
        'stDev': 0.22071344421516914,
        'sampleCount': 647647,
        'noDataCount': 144372}}}}}},
  {'interval': {'from': '2021-08-10T00:00:00Z', 'to': '2021-08-11T00:00:00Z'},
   'outputs': {'ndvi': {'bands': {'B0': {'stats': {'min': -0.6909090876579285,
        'max': 0.8982226252555847,
        'mean': 0.6302106131139007,
        'stDev': 0.28749024291873476,
        'sampleCount': 647647,
        'noDataCount': 220350}}}}}}],
 'status': 'OK'}